# Importing Libreries 

In [1]:
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator
import cv2
import tensorflow as tf
import numpy as np
import torch
import torchvision
from numpy import array


from twilio.rest import Client
# account_sid = 'ACbac462ab71411eb8a4d32fc5686251fa'
# auth_token = '[AuthToken]'
# client = Client(account_sid, auth_token)



# Load, Train Model

In [2]:
model = YOLO('yolov8n-pose.pt')  # load a pretrained YOLOv8n classification model
# model.train(data='coco8-pose.yaml', epochs=3)  # train the model
# model('https://ultralytics.com/images/bus.jpg')  # predict on an image

# Enable GPU 

In [3]:

tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# YOLO Fall Detection 


In [5]:
#-----------------------------------------------------------pose estimation model---------------------------------------------
# Define the video capture device

# cap = cv2.VideoCapture("video (2).avi")
source ="video (1).avi" 
# source ="sleep.mp4" 
cap = cv2.VideoCapture(source)
mog2 = cv2.createBackgroundSubtractorMOG2()



#------------------------Saving the output to the server side--------------------------------------------
#----the incoder-------
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#----------------------------------------
#--------encoding the output streams--------
# THE public video with privacy..
result_Public = cv2.VideoWriter('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\OUT\\public_skelton.mp4', fourcc, 10, (640,480))
# THE private video
result_private = cv2.VideoWriter('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\OUT\\private_RGB.mp4', fourcc, 10, (640,480))
#---------------------------------------------------------------------------------------------------------


#----------------------
fall_counter = 0
#----------------------

c_out_counter = 0

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    # Read a frame from the video stream
    ret, frame = cap.read()
    frame = cv2.resize(frame,(640,480))

    
    # Stop the program if no frame is read
    if not ret:
        break
    
    # Use the model to predict the poses in the frame
    results = model.predict(frame,save=True)
    detections = results[0].plot()


    #-------------------------------------MOG-----------------------------
    #     grey = cv2.copyTo(grayImage,frame)
    mog_first = mog2.apply(frame)
    #     cv.imshow('frame',fgmask)
    #------------------------------------------------

    #----------other methods similar to mog----------
 
    mog_detections0 = cv2.copyTo(frame,mog_first)
    #     denoised_2 = cv2.blur(mog_detections0, (15, 15))
    #     grayImage = cv2.cvtColor(mog_detections0, cv2.COLOR_BGR2GRAY)

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteFrame) = cv2.threshold(grayFrame, 70, 255, cv2.THRESH_BINARY)
    #------------------------------------------------------------------------
  
    

    #-----------------retrieving skelaton from YOLO--------------------------
    boxes = results[0].boxes
    #converting the images torch output from yolo to list
    torch_to_list = boxes.xywh.tolist()

    #cdiminsion of the torch array 
    a = array(torch_to_list)
    #------------------------------------------------------------------------


    # Create a copy of the frame without the boundary boxes
    frame_copy = frame.copy()
    #------------------------------------------------------



    #----------------------------------DETECTING FALLS------------------------------------
    # If the distance to the ground is below a threshold, then the person has fallen
    if  a.shape == (1,4):
        distance_to_ground = torch_to_list[0][3] #Getting The Hight of boundry box
        keypoints_width = torch_to_list[0][2] ##Getting The width of boundry box
        
        #-----------handling sleeping case-----------------------------
        if distance_to_ground < keypoints_width and source !="sleep.mp4" :

            print("Fall detected!")
            fall_counter = fall_counter + 1

    #----------------------------messaging send, Request--------------------------------
#             if fall_counter == 5:
#                 client.messages.create(body = "Fall Detected, Please check your Application..",from_ = '+13855262860', to='+201014454866')

    #--------------------------------------------------------------------------------------
    #----------------------------if the person is not sleeping-----------------------------
            for r in results:
                annotator = Annotator(frame)

                boxes = r.boxes
                for box in boxes:
                        b = box.xyxy[0]  # top, left, bottom, right) format
                        c = box.cls
                        annotator.box_label(b, "Fall Detected")

                frame = annotator.result()  


        else:
            for r in results:
                annotator = Annotator(frame)

                boxes = r.boxes
                for box in boxes:
                        b = box.xyxy[0]  #(top, left, bottom, right) format
                        c = box.cls
                        annotator.box_label(b, "Normal")

                frame = annotator.result()  
    #---------------------------------------------------------------------------------------



    #-----------------------------------output_streams--------------------------------------
    #--------------------------cashing---------------------
    #sending the image to the cache to prevent np problem 
    cv2.imwrite('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\Cache\\temp.jpg', blackAndWhiteFrame)
    public_stream = cv2.imread('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\Cache\\temp.jpg')

    
    #------------------------------------------------------
    #sending the output video to the app
 
    result_Public.write(public_stream) 
    result_private.write(frame_copy)
    
    #---------------------------------------------------------------------------------
    
    
    cv2.imshow('Pose estimation Fall Detection', frame)
    cv2.imshow('skeleton ', detections)
    cv2.imshow('MOG ', blackAndWhiteFrame)
    
    cv2.waitKey(1) == ord('q'):
    break



    # Exit the program if the user presses the 'q' key
#     if c_out_counter == 100:
#         break
#     else: 
#         print(c_out_counter) 
#         c_out_counter = c_out_counter + 1


# Release the video capture device and close the display window
result_Public.release()
result_private.release()
cap.release()
cv2.destroyAllWindows()
#----------------------------------------------------------------------------------------------------------------------------

SyntaxError: invalid syntax (1648041411.py, line 146)

In [ ]:
#-----------------------------------------------------------pose estimation model---------------------------------------------
# Define the video capture device

# cap = cv2.VideoCapture("video (2).avi")
# source ="video (1).avi" 
source ="sleep.mp4" 
cap = cv2.VideoCapture(source)
mog2 = cv2.createBackgroundSubtractorMOG2()



#------------------------Saving the output to the server side--------------------------------------------
#----the incoder-------
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#----------------------------------------
#--------encoding the output streams--------
# THE public video with privacy..
result_Public = cv2.VideoWriter('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\OUT\\public_skelton.mp4', fourcc, 10, (640,480))
# THE private video
result_private = cv2.VideoWriter('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\OUT\\private_RGB.mp4', fourcc, 10, (640,480))
#---------------------------------------------------------------------------------------------------------


#----------------------
fall_counter = 0
#----------------------

c_out_counter = 0

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    # Read a frame from the video stream
    ret, frame = cap.read()
    frame = cv2.resize(frame,(640,480))

    
    # Stop the program if no frame is read
    if not ret:
        break
    
    # Use the model to predict the poses in the frame
    results = model.predict(frame,save=True)
    detections = results[0].plot()


    #-------------------------------------MOG-----------------------------
    #     grey = cv2.copyTo(grayImage,frame)
    mog_first = mog2.apply(frame)
    #     cv.imshow('frame',fgmask)
    #------------------------------------------------

    #----------other methods similar to mog----------
 
    mog_detections0 = cv2.copyTo(frame,mog_first)
    #     denoised_2 = cv2.blur(mog_detections0, (15, 15))
    #     grayImage = cv2.cvtColor(mog_detections0, cv2.COLOR_BGR2GRAY)

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteFrame) = cv2.threshold(grayFrame, 70, 255, cv2.THRESH_BINARY)
    #------------------------------------------------------------------------
  
    

    #-----------------retrieving skelaton from YOLO--------------------------
    boxes = results[0].boxes
    #converting the images torch output from yolo to list
    torch_to_list = boxes.xywh.tolist()

    #cdiminsion of the torch array 
    a = array(torch_to_list)
    #------------------------------------------------------------------------


    # Create a copy of the frame without the boundary boxes
    frame_copy = frame.copy()
    #------------------------------------------------------



    #----------------------------------DETECTING FALLS------------------------------------
    # If the distance to the ground is below a threshold, then the person has fallen
    if  a.shape == (1,4):
        distance_to_ground = torch_to_list[0][3] #Getting The Hight of boundry box
        keypoints_width = torch_to_list[0][2] ##Getting The width of boundry box
        
        #-----------handling sleeping case-----------------------------
        if distance_to_ground < keypoints_width and source !="sleep.mp4" :

            print("Fall detected!")
            fall_counter = fall_counter + 1

    #----------------------------messaging send, Request--------------------------------
#             if fall_counter == 5:
#                 client.messages.create(body = "Fall Detected, Please check your Application..",from_ = '+13855262860', to='+201014454866')

    #--------------------------------------------------------------------------------------
    #----------------------------if the person is not sleeping-----------------------------
            for r in results:
                annotator = Annotator(frame)

                boxes = r.boxes
                for box in boxes:
                        b = box.xyxy[0]  # top, left, bottom, right) format
                        c = box.cls
                        annotator.box_label(b, "Fall Detected")

                frame = annotator.result()  


        else:
            for r in results:
                annotator = Annotator(frame)

                boxes = r.boxes
                for box in boxes:
                        b = box.xyxy[0]  #(top, left, bottom, right) format
                        c = box.cls
                        annotator.box_label(b, "Normal")

                frame = annotator.result()  
    #---------------------------------------------------------------------------------------



    #-----------------------------------output_streams--------------------------------------
    #--------------------------cashing---------------------
    #sending the image to the cache to prevent np problem 
    cv2.imwrite('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\Cache\\temp.jpg', blackAndWhiteFrame)
    public_stream = cv2.imread('C:\\Users\\kools\\Downloads\\Grad\\YOLO V8\\Cache\\temp.jpg')

    
    #------------------------------------------------------
    #sending the output video to the app
 
    result_Public.write(public_stream) 
    result_private.write(frame_copy)
    
    #---------------------------------------------------------------------------------
    
    
    cv2.imshow('Pose estimation Fall Detection', frame)
    cv2.imshow('skeleton ', detections)
    cv2.imshow('BW ', blackAndWhiteFrame)
    cv2.imshow('MOG ', mog_first)
    
     # Exit the program if the user presses the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break



    # Exit the program if the user presses the 'q' key
#     if c_out_counter == 100:
#         break
#     else: 
#         print(c_out_counter) 
#         c_out_counter = c_out_counter + 1


# Release the video capture device and close the display window
result_Public.release()
result_private.release()
cap.release()
cv2.destroyAllWindows()
#----------------------------------------------------------------------------------------------------------------------------


0: 480x640 (no detections), 137.6ms
Speed: 4.0ms preprocess, 137.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 168.5ms
Speed: 19.0ms preprocess, 168.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 148.6ms
Speed: 3.0ms preprocess, 148.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 164.9ms
Speed: 7.0ms preprocess, 164.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 161.0ms
Speed: 6.0ms preprocess, 161.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 139.9ms
Speed: 4.1ms preprocess, 139.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Resu

Results saved to runs\pose\predict22

0: 480x640 1 person, 131.0ms
Speed: 2.0ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 133.0ms
Speed: 7.0ms preprocess, 133.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 131.9ms
Speed: 4.0ms preprocess, 131.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 132.5ms
Speed: 4.5ms preprocess, 132.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 131.6ms
Speed: 2.5ms preprocess, 131.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 150.0ms
Speed: 3.0ms preprocess, 150.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results sa

Speed: 4.0ms preprocess, 194.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 2 persons, 198.5ms
Speed: 4.5ms preprocess, 198.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 216.0ms
Speed: 7.0ms preprocess, 216.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 189.0ms
Speed: 4.0ms preprocess, 189.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 192.5ms
Speed: 3.5ms preprocess, 192.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 179.0ms
Speed: 3.0ms preprocess, 179.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 181.0ms
Speed: 4.

Results saved to runs\pose\predict22

0: 480x640 2 persons, 135.0ms
Speed: 3.0ms preprocess, 135.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 3 persons, 129.0ms
Speed: 4.0ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 131.0ms
Speed: 3.0ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 132.0ms
Speed: 3.0ms preprocess, 132.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 161.0ms
Speed: 4.0ms preprocess, 161.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 133.0ms
Speed: 4.0ms preprocess, 133.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results 

Speed: 2.5ms preprocess, 133.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 132.5ms
Speed: 3.0ms preprocess, 132.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 130.0ms
Speed: 2.5ms preprocess, 130.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 132.9ms
Speed: 3.0ms preprocess, 132.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 133.0ms
Speed: 4.0ms preprocess, 133.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 (no detections), 169.0ms
Speed: 24.0ms preprocess, 169.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 137.0ms
Sp

Results saved to runs\pose\predict22

0: 480x640 1 person, 138.0ms
Speed: 4.0ms preprocess, 138.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 129.0ms
Speed: 3.5ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 133.0ms
Speed: 4.0ms preprocess, 133.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 149.8ms
Speed: 2.3ms preprocess, 149.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 131.0ms
Speed: 3.0ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 130.0ms
Speed: 4.0ms preprocess, 130.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results sa

Speed: 4.0ms preprocess, 175.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 292.0ms
Speed: 3.0ms preprocess, 292.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 176.6ms
Speed: 5.0ms preprocess, 176.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 306.7ms
Speed: 3.0ms preprocess, 306.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 175.9ms
Speed: 5.0ms preprocess, 175.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 173.0ms
Speed: 3.0ms preprocess, 173.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 188.9ms
Speed: 4.0

Results saved to runs\pose\predict22

0: 480x640 1 person, 193.0ms
Speed: 4.0ms preprocess, 193.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 206.0ms
Speed: 4.0ms preprocess, 206.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 202.0ms
Speed: 4.0ms preprocess, 202.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 204.0ms
Speed: 6.0ms preprocess, 204.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 1 person, 161.0ms
Speed: 9.0ms preprocess, 161.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\pose\predict22

0: 480x640 2 persons, 131.0ms
Speed: 4.0ms preprocess, 131.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results s